In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2023-07-25


Found rundate 2023-07-24 10:00:02 < 2023-07-25 at commit 822224fe; returning commit ad85c69


'2023-07-25 10:00:03'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2023-07-25 10:00:03, NJSP was reporting 301 YTD deaths
Current YTD Deaths (2024-07-25 10:38:05-04:00): 346
Previous year YTD Deaths (adjusted; 2023-07-25 10:00:03): 301.67780668288236
Projected 2024 total: 676.6191284418737
14.7% change


FAUQStats cache miss: 2023, Tue Jul 25 10:00:03 EDT 2023


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,HIGHWAY,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,INJURIES,STREET,dt
ACCID,,,,,,,,,,,,,,
12249,07,Essex,0714,Newark City,NaN,Mulberry St,1.0,0.0,0.0,1.0,0.0,NaN,Mulberry St,2023-01-01 01:49:00-05:00
12251,01,Atlantic,0112,Hamilton Twsp,322,State Highway 322 MP 46.4,1.0,1.0,0.0,0.0,0.0,0.0,NaN,2023-01-02 09:26:00-05:00
12253,12,Middlesex,1217,Piscataway Twsp,NaN,South Randolphville Rd,1.0,1.0,0.0,0.0,0.0,0.0,South Randolphville Rd,2023-01-02 19:41:00-05:00
12254,02,Bergen,0221,Garfield City,NaN,Outwater Ln,1.0,0.0,0.0,1.0,0.0,0.0,Outwater Ln,2023-01-03 20:52:00-05:00
12267,07,Essex,0714,Newark City,NaN,Washington St,1.0,0.0,0.0,1.0,0.0,NaN,Washington St,2023-01-04 06:21:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12574,17,Salem,1713,Carneys Point Twsp,95,New Jersey Turnpike MP 2.4,1.0,1.0,0.0,0.0,0.0,0.0,NaN,2023-07-22 09:43:00-04:00
12577,16,Passaic,1614,Wayne Twsp,NaN,Rt 80 E Ramp from Rt 23 S,1.0,1.0,0.0,0.0,0.0,NaN,Rt 80 E Ramp from Rt 23 S,2023-07-23 00:19:00-04:00
12571,01,Atlantic,0101,Absecon City,30,State Highway 30,1.0,0.0,1.0,0.0,0.0,NaN,NaN,2023-07-23 03:25:00-04:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,15,1,11,2,29
Bergen,10,3,10,1,22
Burlington,19,4,2,1,26
Camden,12,3,10,0,25
Cape May,2,0,0,0,2
Cumberland,8,2,2,0,11
Essex,9,3,14,0,26
Gloucester,7,2,0,0,9
Hudson,3,2,1,0,5


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

56.4% through the year, 43.6% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,9,18,15
Bergen,driver,12,20,10
Burlington,driver,12,26,19
Camden,driver,9,18,12
Cape May,driver,3,4,2
...,...,...,...,...
Salem,crashes,5,11,8
Somerset,crashes,8,21,9
Sussex,crashes,1,6,7


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,55,4,27,3,21
Bergen,39,2,17,6,17
Burlington,53,2,38,7,4
Camden,57,2,23,6,29
Cape May,4,0,3,0,0
Cumberland,20,0,16,3,3
Essex,58,0,17,5,39
Gloucester,17,0,13,3,1
Hudson,12,1,7,4,3


In [9]:
projected.to_csv(PROJECTED_CSV)